In [1]:
import pandas as pd
from shapely.geometry import Point
import hydrobr
import geopandas as gpd

In [2]:
# Carrega dados das estações de chuvas usadas.
df_chuvas = pd.read_csv(r'C:\Users\pedro\Documents\Pibic 21-22\Tratamento-fluviometrico\Dados\chuvas.csv', index_col=0, parse_dates=True)

In [3]:
# Obtem a média de chuva por estação.
df_medias_k = pd.DataFrame(df_chuvas.mean(), columns=['Average'])
df_medias_k

,Average
00536018,1.997889
00536025,2.272252
00536031,1.783850
00636001,1.265214
00636003,1.835047
...,...
00738077,2.011662
00738078,1.896629
00738079,1.956834
00738080,2.144400


In [4]:
# Obtem códigos das estações plu
codigos = df_chuvas.columns

In [5]:
# Cria df contendo as estações plu de interesse
lista_estacoes = hydrobr.get_data.ANA.list_prec_stations()
lista_estacoes = lista_estacoes.loc[lista_estacoes['Code'].isin(codigos)]

In [6]:
# Cria geometria para as estações
geometry = [Point(xy) for xy in zip(lista_estacoes['Longitude'], lista_estacoes['Latitude'])]
lista_estacoes = gpd.GeoDataFrame(lista_estacoes, geometry=geometry)

In [7]:
# Carrega os dados das bacias
leste = gpd.read_file(r"C:\Users\pedro\Documents\Pibic 21-22\Tratamento-fluviometrico\Dados\Bacia_Leste.shp")
oeste = gpd.read_file(r"C:\Users\pedro\Documents\Pibic 21-22\Tratamento-fluviometrico\Dados\Bacia_oeste.shp")

In [8]:
# Dividi estações da região leste da bacia
pip_mask_leste = lista_estacoes.within(leste.loc[0, 'geometry'])
pip_data_leste = lista_estacoes.loc[pip_mask_leste]

In [9]:
# Dividi estações da região oeste da bacia
pip_mask_oeste = lista_estacoes.within(oeste.loc[0, 'geometry'])
pip_data_oeste = lista_estacoes.loc[pip_mask_oeste]

In [10]:
# Obtem codigos das estações plu do leste e oeste
codigos_leste = pip_data_leste['Code'].to_list()
codigos_oeste = pip_data_oeste['Code'].to_list()

In [11]:
# Define o número do cluster por leste/oeste
df_medias_k['Cluster'] = 0
df_medias_k.loc[codigos_leste, 'Cluster'] = 1
df_medias_k.loc[codigos_oeste, 'Cluster'] = 2

In [12]:
# Exporta tabela
df_medias_k.to_excel(r'C:\Users\pedro\Documents\Pibic 21-22\Tratamento-fluviometrico\Dados\medias_cluster.xlsx')